<a href="https://colab.research.google.com/github/caio-torres-seares/assistente-virtual/blob/main/assistente_virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gTTS
!pip install playsound
!pip install SpeechRecognition
!pip install wikipedia-api

In [ ]:
# Resolvendo bug do pyaudio
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (1,022 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126435 files and directories currently installed.)
P

In [57]:
from gtts import gTTS
from IPython.display import Audio, display
import datetime
import webbrowser
import pytz
import playsound
import os
import speech_recognition as sr
from webbrowser import open as open_browser
import wikipediaapi as wikipedia

In [60]:
USAR_AUDIO = False

def verificar_audio_disponivel():
    """Verifica se audio está disponível no sistema"""
    global USAR_AUDIO
    try:
        # Testar se microfone está disponível
        r = sr.Recognizer()
        mic_list = sr.Microphone.list_microphone_names()
        if mic_list:
            # Testar se consegue acessar o microfone
            with sr.Microphone() as source:
                pass
            USAR_AUDIO = True
            print("✓ Sistema de áudio detectado - Modo voz ativado")
            return True
        else:
            print("⚠ Sistema de áudio não detectado - Modo texto ativado")
            return False
    except Exception as e:
        print(f"⚠ Audio não disponível: {e}")
        print("Modo apenas texto ativado")
        USAR_AUDIO = False
        return False

In [61]:
import time

def falar(texto):
    print(f"Assistente: {texto}")
    try:
        # Cria o áudio usando o Google Text-to-Speech em português do Brasil
        tts = gTTS(text=texto, lang='pt')

        filename = 'voice.mp3'
        tts.save(filename)

        display(Audio(filename, autoplay=True))

        # Estimativa de tempo para o áudio terminar (aprox. 0.5s por palavra)
        duracao = max(2, len(texto.split()) * 0.5)
        time.sleep(duracao)

        os.remove(filename)
    except Exception as e:
        print(f"Ocorreu um erro ao falar: {e}")

In [62]:
def ouvir_microfone():
    r = sr.Recognizer()

    with sr.Microphone() as source:
        print("Ajustando para ruído ambiente. Por favor, aguarde.")
        r.adjust_for_ambient_noise(source, duration=1)
        print("Pode falar! Estou ouvindo...")

        # Ouve o áudio do microfone
        audio = r.listen(source)

        try:
            # Tenta reconhecer a fala usando a API do Google
            texto = r.recognize_google(audio, language='pt-BR')
            print(f"Você disse: {texto}")
            return texto.lower() # Retorna o texto em minúsculas
        except sr.UnknownValueError:
            falar("Desculpe, não consegui entender o que você disse.")
            return ""
        except sr.RequestError as e:
            falar(f"Não foi possível solicitar resultados do serviço de reconhecimento de fala; {e}")
            return ""

In [63]:
def processar_comandos(comando):
    if 'pesquise por' in comando:
        termo = comando.replace('pesquise por', '').strip()
        falar(f"Buscando por '{termo}' na Wikipedia.")
        try:
            wikipedia.set_lang('pt')
            resultado = wikipedia.summary(termo, sentences=2)
            falar("De acordo com a Wikipedia...")
            falar(resultado)
        except wikipedia.exceptions.PageError:
            falar(f"Desculpe, não encontrei a página para '{termo}'.")
        except wikipedia.exceptions.DisambiguationError:
            falar(f"Existem muitas páginas para '{termo}'. Por favor, seja mais específico.")

    elif 'abrir o youtube' in comando:
        falar("Abrindo o YouTube.")
        open_browser("https://www.youtube.com")

    elif 'procure no youtube por' in comando:
        termo = comando.replace('procure no youtube por', '').strip()
        falar(f"Procurando por '{termo}' no YouTube.")
        url = f"https://www.youtube.com/results?search_query={termo.replace(' ', '+')}"
        open_browser(url)

    elif 'farmácia mais próxima' in comando:
        falar("Mostrando a localização da farmácia mais próxima no mapa.")
        url = "https://www.google.com/maps/search/farmácia+mais+próxima"
        open_browser(url)

    elif 'encerrar' in comando or 'parar' in comando:
        falar("Até logo!")
        return True # Sinaliza para o loop principal terminar

    else:
        falar("Comando não reconhecido. Tente novamente.")

    return False

In [ ]:
# Verificar disponibilidade de áudio
verificar_audio_disponivel()

if USAR_AUDIO:
    falar("Olá! Como posso ajudar?")
else:
    falar("ASSISTENTE VIRTUAL - MODO TEXTO")
    print("\n" + "="*50)
    print("ASSISTENTE VIRTUAL - MODO TEXTO")
    print("="*50)
    print("Assistente: Olá! Como posso ajudar?")
    print("\nComandos disponíveis:")
    print("- 'pesquise por [termo]'")
    print("- 'abrir o youtube'")
    print("- 'procure no youtube por [termo]'")
    print("- 'farmácia mais próxima'")
    print("- 'encerrar' ou 'parar'")
    print("="*50 + "\n")

while True:
    try:
        if USAR_AUDIO:
          comando = ouvir_microfone()
        else:
          comando = input("Você: ")
        if comando:
            if processar_comandos(comando):
                break # Encerra o loop se a função retornar True
    except KeyboardInterrupt:
        if USAR_AUDIO:
            falar("Encerrando assistente.")
        else:
            print("\nAssistente: Encerrando assistente.")
        break


⚠ Sistema de áudio não detectado - Modo texto ativado
Assistente: ASSISTENTE VIRTUAL - MODO TEXTO



ASSISTENTE VIRTUAL - MODO TEXTO
Assistente: Olá! Como posso ajudar?

Comandos disponíveis:
- 'pesquise por [termo]'
- 'abrir o youtube'
- 'procure no youtube por [termo]'
- 'farmácia mais próxima'
- 'encerrar' ou 'parar'

Você: abrir o youtube
Assistente: Abrindo o YouTube.


Você: farmácia mais próxima
Assistente: Mostrando a localização da farmácia mais próxima no mapa.
